In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import keras
from torch.utils.data import DataLoader,TensorDataset
import matplotlib.pyplot as plt
import xlsxwriter
def savexlsx(data,name):
    workbook=xlsxwriter.Workbook('{}.xlsx'.format(name))
    worksheet=workbook.add_worksheet()
    hang = data.shape[0]
    lie = data.shape[1]
    for col in range(int(hang)):
        for row in range(int(lie)):
            worksheet.write(col,row,data[col,row])
    workbook.close()
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch':[], 'epoch':[]}
        self.accuracy = {'batch':[], 'epoch':[]}
        self.val_loss = {'batch':[], 'epoch':[]}
        self.val_acc = {'batch':[], 'epoch':[]}

    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('acc'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_acc'))

    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('acc'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_acc'))

    def loss_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        plt.figure()
        # acc
        plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
        # loss
        plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # val_acc
            plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')
        plt.legend(loc="upper right")
        plt.show()

Training Phase

In [ ]:
dataset = pd.read_excel('spectrums and proportion.xlsx', header = None)

In [ ]:
orignal_data = np.array(dataset)

In [ ]:
orignal_data.shape

In [ ]:
spectrums = orignal_data[:,:473]

In [ ]:
proportion = orignal_data[:,473]

In [ ]:
train_x, valid_x, train_y, valid_y = train_test_split(spectrums, proportion, test_size=0.1)

In [ ]:
t_y = train_y.reshape(-1,1)
v_y = valid_y.reshape(-1,1)

In [ ]:
savexlsx(train_x,'train_x')
savexlsx(valid_x,'valid_x')
savexlsx(train_y,'train_y')
savexlsx(valid_y,'valid_y')

In [ ]:
from keras import regularizers
from keras.layers import Reshape,Add,Multiply,merge,Input,Dense, Activation,Dense,Dropout,Flatten,BatchNormalization,Conv1D
from keras.layers import MaxPooling1D,AveragePooling1D,LeakyReLU,GlobalAveragePooling1D
from keras.models import Sequential,Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

In [ ]:
def Res_block(X,filters):
    
    F1,F2 = filters
 
    X_shortcut = X
 
    X = Conv1D(filters = F1, kernel_size = 1, strides = 1)(X)
    X = Dropout(0.5)(X)
    X = Activation('relu')(X)
    X = Conv1D(filters = F2, kernel_size = 3, strides = 1, padding = 'same')(X)
    
    X_shortcut = Conv1D(F2, kernel_size = 1, strides = 1)(X_shortcut)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
 
    return X

In [ ]:
def LPnet(input_shape = (473,), classes = 1):
 
    X_input = Input(input_shape)
    X1 = Dense(473, activation = 'relu', name = 'dense1')(X_input)
    X1 = Dropout(0.5)(X1)
    X1 = Dense(256, activation = 'relu', name = 'dense2')(X1)
    X1 = Dropout(0.5)(X1)
    X1 = Dense(473, name = 'dense3')(X1)
    X = Multiply()([X1, X_input])
    X = Reshape((473, 1))(X)
    12
    X = Res_block(X,filters=[256,128])
    X = Res_block(X,filters=[64,32])
    X = Res_block(X,filters=[32,16])

    X = AveragePooling1D(3, name = 'avg_pool')(X)
    X = Flatten()(X)
    X = Dense(128, activation = 'relu')(X)
    X = Dense(64, activation = 'relu')(X)
    X = Dense(classes)(X)
    model = Model(inputs = X_input, outputs = X, name = 'LPNet')
    return model

In [ ]:
model = LPnet()

In [ ]:
model.summary()

In [ ]:
model.compile(loss='mse', optimizer=Adam(lr=0.01), metrics=['accuracy'])  

In [ ]:
history=LossHistory()

In [ ]:
model_names = "model_epoch:{epoch:02d}-val_loss:{val_loss:.4f}.hdf5"

In [ ]:
save_model = ModelCheckpoint(filepath= model_names, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [ ]:
record_h=model.fit(train_x, train_y, epochs=3000,batch_size=256, validation_data=(valid_x, valid_y), callbacks=[history,save_model])

In [ ]:
loss = record_h.history['loss']
val_loss = record_h.history['val_loss']
acc = record_h.history['acc']
val_acc = record_h.history['val_acc']

In [ ]:
log_data = np.stack((loss, val_loss, acc,val_acc), axis=1)

In [ ]:
savexlsx(log_data,'log')

# Testing Phase

In [ ]:
from keras.models import load_model

In [ ]:
model_p = load_model('model_epoch_xxx-val_loss_xxx.hdf5')

In [ ]:
test_x = pd.read_excel('test_x.xlsx', header = None)

In [ ]:
test_x = np.array(test_x)

In [ ]:
test_result = model_p.predict(test_x)
savexlsx(test_result,'test_result')

In [ ]:
train_results = model_p.predict(train_x)

In [ ]:
savexlsx(train_results,'train_results')

In [ ]:
valid_results = model_p.predict(valid_x)

In [ ]:
savexlsx(valid_results,'valid_results')

# Attention Extraction Phase

In [ ]:
model_p.summary()

In [ ]:
layer_name = 'Dense3' # alterable name 
intermediate_layer_model = Model(inputs=model_p.input,outputs=model_p.get_layer(layer_name).output)#创建的新模型
output1 = intermediate_layer_model.predict(train_x)
output2 = intermediate_layer_model.predict(valid_x)
output3 = intermediate_layer_model.predict(test_x)

In [ ]:
savexlsx(output1,'train_attention_value')
savexlsx(output2,'valid_attention_value')
savexlsx(output3,'test_attention_value')

In [ ]:
layer_name = 'multiply_1' # alterable name
intermediate_layer_model = Model(inputs=model_p.input,outputs=model_p.get_layer(layer_name).output)#创建的新模型
output4 = intermediate_layer_model.predict(valid_x)
output5 = intermediate_layer_model.predict(train_x)
output6 = intermediate_layer_model.predict(test_x)

In [ ]:
savexlsx(output4,'train_attention_out')
savexlsx(output5,'valid_attention_out')
savexlsx(output6,'test_attention_out')